# Data Cleaning in Pandas

![](https://media.giphy.com/media/AhAysobj49aqQ/giphy.gif)

## Learning goals: To apply pandas data cleaning methods to animal shelter data.
![austin](http://www.austintexas.gov/sites/default/files/aac_logo.jpg)
 

### Agenda:
- Understand why data cleaning is important
- Review ways to read data into a pandas dataframe
- Apply pandas methods to inspect our data
- Clean our data using pandas methods

### Why is data cleaning important and how does it fit into the data science process?

Remember this CRISP-DM Model?

<img src='https://storage.ning.com/topology/rest/1.0/file/get/2808314343?profile=RESIZE_480x480' width='500'>

#### ACTIVITY

With your group, read [this article](https://counting.substack.com/p/data-cleaning-is-analysis-not-grunt?utm_campaign=Data_Elixir&utm_source=Data_Elixir_303) (focus on the sections "Data Cleaning IS Analysis" through "Cleaning your data allows you to know your data" and disucuss:

- Why is data cleaning important?
- What questions should you be thinking about as your clean your data?
- How can data cleaning help you in your analysis?

### Get and inspect data

In [ ]:
import pandas as pd

The data from the [Austin Animal Shelter](http://www.austintexas.gov/department/aac) is hosted in these locations:

**Intakes**:
https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm <br>
**Outcomes**: https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Outcomes/9t4d-g238

We will read it into our notebook using [pd.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [ ]:
outcomes = pd.read_csv('./data/Austin_Animal_Center_Outcomes.csv')

Let's do the same for intakes!

In [ ]:
intakes = pd.read_csv('./data/Austin_Animal_Center_Intakes.csv')

### Inspect data
#### Check top and bottom of dataset

We can use the `.head()` method to view the first few rows of our dataframe.  

Note: by default the function returns the first 5 rows but you can view more or less by specifying the number of rows you want to view inside the () like this `.head(20)`.

In [ ]:
outcomes.head()

Similarly we can view the bottom of our dataframe by using the `.tail()` method.

In [ ]:
outcomes.tail(10)

In [ ]:
type(outcomes)

It's important that we know it's a `DataFrame` because now, given the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), we can always expect answers on any dataset we load in. 

#### What's the length and width of our dataframe?

In [ ]:
outcomes.shape

The `.shape` attribute tells us how many rows and columns in our dataframe. Our outcomes dataset has 108,519 rows and 12 columns of data.

#### Question:  What does a row of data represent in this dataset?  What are some things we should consider when we are performing analysis on this dataset?

#### Get column names

We might also want to examine just the names of each column in our dataframe.  We can do this by using the `.columns` attribute.

In [ ]:
outcomes.columns

**Columns** in a dataframe on an individual level are `Series` objects <br>
To access an individual column, the easiest way to to use `.` notation:<br>
`outcomes.Name`

In [ ]:
outcomes.Name

If your column name has spaces in it the `.` notation will not work but you can use `[]` to access those columns.

In [ ]:
outcomes['Outcome Type']

#### Check data type of each column
Type of the data (integer, float, Python object, etc.)

In [ ]:
outcomes.dtypes

#### Get data type *and* an idea of how many missing values
Which columns have missing data?

In [ ]:
outcomes.info()

As an alternative we can look at the sum of all missing values by chaining the `.isna()` function which is a boolean with the `.sum()` function.

In [ ]:
outcomes.isna().sum()

### Your Turn!

### Apply to `intakes`

Now, for the `intakes` dataset. How does it compare to `outcomes`?
- does it have the same number of observations?
- same column  names?
- do rows in data represent the same level of information?
- are the datatypes the same or different?
- what about missing data?

In [ ]:
# your code here

#### Now let's find the age of the animals in the shelter!

#### This should be easy, we have 'Age upon Outcome' in our outcomes dataframe

In [ ]:
outcomes['Age upon Outcome'].mean()

### Wait! Something went wrong!
What happened? Why?

We are going to need to struggle through some data cleaning

![panda struggle](img/panda_struggle.gif)

## Data Cleaning

**First step**: make the column names easier to work with

Going to use `str`, `lower`, and [`replace`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.replace.html) to make our lives easier.

In [ ]:
outcomes.columns

In [ ]:
outcomes.columns = outcomes.columns.str.lower()
outcomes.columns

In [ ]:
outcomes.columns = outcomes.columns.str.replace(' ', '_')
outcomes.columns

### Your Turn!

#### Apply the above cleaning to intakes!

In [ ]:
# your code here

#### **Why** care about that?
Because now I can use `tab` to find column names.<br>

#### How many of each type of animals are in the outcomes dataset?

We can use the [`value_counts()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html) function to get counts for each value in the animal types column in outcomes.<br>


In [ ]:
outcomes.animal_type.value_counts()

### Let's see the unique values of age

In [ ]:
outcomes.age_upon_outcome.value_counts()

#### What's the challenge with these numbers?

#### What could we use instead?

#### Steps needed:
- convert dates to correct date types
- create a new age variable subtracting dates
- drop the original age variable

### Converting dtypes

Okay, going to use a [`apply`](https://pandas.pydata.org/pandas-docs/version/0.18/generated/pandas.Series.apply.html) and a [`lambda`](https://www.w3schools.com/python/python_lambda.asp) function. 



It's getting exciting, now!


`apply`, `map`, and `applymap`
<img src='https://miro.medium.com/max/1796/1*deCRAl5DuNZ1a0TNGKYrNQ.png' width='500'>


#### Anonymous Functions (Lambda Abstraction)

Simple functions can be defined right in the function call. This is called 'lambda abstraction'; the function thus defined has no name and hence is "anonymous".

It looks like the `datetime` column contains the date and time the outcome occured. Let's create a new column called `date_o` where we copy the `datetime` column.

In [ ]:
outcomes['date_o'] = outcomes.datetime
outcomes.head()

Great we added that new column!  But we don't really care about the time of day the outcome occured.  We only care about the date!  Let's use a lambda function to slice that datetime!

In [ ]:
outcomes['date_o'] = outcomes.date_o.apply(lambda x: x[:10])
outcomes.head()

Awesome!  We shortened the date.  But it's still being read as an object datatype.

#### Using [`to_datetime`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) we can convert this to a datetime format where we can then calculate the age of the animal.

In [ ]:
# convert date formats
outcomes['date_o'] =  pd.to_datetime(outcomes['date_o'], format='%m/%d/%Y')
outcomes['dob'] =  pd.to_datetime(outcomes['date_of_birth'], format='%m/%d/%Y')

Check to see if it worked!

In [ ]:
outcomes.head()

In [ ]:
outcomes.dtypes

We did it!<br>
Let's [`drop`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) the variables we will no longer use. 

In [ ]:
outcomes = outcomes.drop(columns=['datetime', 'date_of_birth'] )
outcomes

### Make new variable of age and years_old

In [ ]:
outcomes['age'] = outcomes.date_o - outcomes.dob

In [ ]:
outcomes['years_old'] = outcomes.age.apply(lambda x: x.days/365)
outcomes

In [ ]:
outcomes.dtypes

### NOW try `mean`!

In [ ]:
outcomes.years_old.mean()

#### Great!  What does this mean?  What question about the data have we answered?

### Filtering and sub-setting

What if we want to see the mean age of each type of animal in the shelter?  How would we do that?


We can use a [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) function to help us aggregate and filter.

In [ ]:
outcomes[['animal_type', 'years_old']].groupby(['animal_type']).mean()

### Your turn! 

With your group, convert `datetime`to a datetime object.

In [ ]:
# your code here

## Dealing with Missing values

We saw earlier that we have some missing values in several columns.  Now we need to decide what to do about them.  One option is to __fill__ in the missing value and another option is to __drop__ that missing value.

### Activity

In your group, discuss the pros and cons of filling the missing values vs dropping the missing values for the following columns of our dataframe.

- `name`
- `outcome_type`
- `outcome_subtype`
- `sex_upon_outcome`
- `age_upon_outcome`

For columns in which you feel like it is important to fill the missing data what do you think we should fill these values with?

####  Dropping rows with missing data
Because there are very few (only 3 of the 108,519) rows of data that are missing the `sex_upon_outcome` variable we can drop the rows where this variable is missing and we will only lose less than 1% of that data.  Let's go ahead and drop these rows. 

We will use the `dropna` function to execute this drop. Note:  We will need to use the `subset=` argument to drop missing values in this column only.

In [ ]:
outcomes = outcomes.dropna(subset=['sex_upon_outcome'])
outcomes.shape

#### Filling missing data

Now let's talk about the missing values for name.  If we dropped all these rows we would lose about 32% of our data!  That's a lot! Plus, maybe we want to examine how many of the animals in the shelter don't have names.  Then we would need this information! So instead of dropping those rows let's replace missing names with the string "No name given".

We can use the `.fillna` function to fill in those missing values with our desired string.

In [ ]:
outcomes['name'] = outcomes.name.fillna("No name given")

In [ ]:
outcomes.isna().sum()

#### Great!  We have successfully cleaned up two of columns with missing values!

### Your turn!

In your group, work on cleaning the `outcome_subtype` column and the `age_upon_outcome` column.  Be thoughtful in how you deal with these missing values.  Be able to explain why you made the decisions you did!

In [ ]:
# your code here

#### Now we are rolling!!

![panda roll](img/panda_rolling.gif)

### Further Resources
- Learn from [Wes McKinney himself](https://www.youtube.com/watch?v=_T8LGqJtuGc#action=share) in his "Pandas in 10 minutes video"
- Make the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/index.html) your best friend